In [ ]:
# Vanilla Imports
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gmean
import src.Luminosity.red_tree as red
# Custom Imports
plt.rcParams['text.usetex'] = True
plt.rcParams['figure.dpi'] = 300
plt.rcParams['figure.figsize'] = [5 , 4]
Rsol_to_cm = 6.957e10 # [cm]

num_array = np.arange(2000, 9000, 1000)
snap = 844
m = 6

In [ ]:
lum = np.zeros(len(num_array))
rays_photo_mean = np.zeros(len(num_array))
for i,num in enumerate(num_array):
    print('num ', num)
    lum[i], rays_photo = red.doer_of_thing(snap, m, num) 
    rays_photo_mean[i] = gmean(rays_photo)

In [ ]:
with open('data/red/convergence_m'+ str(m) + '.txt', 'a') as file:
    file.write('# num\n') 
    file.write(' '.join(map(str, num_array)) + '\n')
    file.write('# Lum \n') 
    file.write(' '.join(map(str, lum)) + '\n')
    file.write('# gmean R_{ph} \n') 
    file.write(' '.join(map(str, rays_photo_mean)) + '\n')
    file.close() 

In [ ]:
data = np.loadtxt('data/red/convergence_m6.txt')
num_plot = data[0]
lum_plot = data[1]
photo_plot = data[2]/Rsol_to_cm

In [ ]:
img, ax = plt.subplots(figsize = (10,8))
fig = ax.scatter(num_plot, lum_plot, c = photo_plot, vmin = 54, vmax = 57)
cbar = img.colorbar(fig)
cbar.set_label(r'$\bar{R}_{ph} [R_\odot]$', fontsize = 15)
ax.set_ylabel('$\log_{10}$ L [erg/s]', fontsize = 15)
ax.set_xlabel('Num cells', fontsize = 15)
ax.set_yscale('log')
ax.set_ylim(5e42,2e43)
plt.grid()
plt.savefig('Final plot/convergence' + str(snap) + '.png')
plt.show()